# BREWSCRAPE
To scrape the recipes, we will be using brewersfriend recipe search page: https://www.brewersfriend.com/search/.
The first step is to scrape all the urls provided for each style provided in the aformentioned link. After these 
are acquired, we can scrape the contents of each individual link.
## Explore Styles
First let's explore the styles that are available. Since the table of links is dynamic and you can't navigate to 
specific pages I'm choosing to scrape the links by iterating though each individual style. That way, if there
are bugs/ errors, we can start from the most recent successfully fully scraped style instead of starting on
page 1 of all styles (5000+ pages total).

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import time
import sys
import pandas as pd
import random
import re

In [2]:
#get list of styles we will iterate through to scrape urls
url = 'https://www.brewersfriend.com/search/'
hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
req = Request(url, headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, "lxml")

brewStyle_container = soup.find('select',id='styleid')
brewStyles = []
for i in range(0,len(brewStyle_container.find_all('option'))-1): #222
    brewStyles.append(brewStyle_container.find_all('option')[i+1].text)

In [3]:
#note there are multiples of some styles 
#this is due to an update that allowed the ingredients in newer recipes to have links to their usage/statistics
print('Number of styles (includes multiples):',len(brewStyles))
print(brewStyles[0:10])

print('\n','Number of unique styles:',len(set(brewStyles)))
print(list(set(brewStyles[0:10])))

multiplesList = brewStyles
for x in list(set(brewStyles)):
    multiplesList.remove(x)

print('\n','Number of styles with multiples:',len(multiplesList))
print(multiplesList)

#get brewStyle list again
brewStyles = []
for i in range(0,len(brewStyle_container.find_all('option'))-1): #222
    brewStyles.append(brewStyle_container.find_all('option')[i+1].text)

brewStyles = sorted(list(set(brewStyles)))

Number of styles (includes multiples): 221
['Altbier', 'Alternative Grain Beer', 'Alternative Sugar Beer', 'American Amber Ale', 'American Amber Ale', 'American Barleywine', 'American Barleywine', 'American Brown Ale', 'American Brown Ale', 'American IPA']

 Number of unique styles: 181
['American Barleywine', 'Alternative Grain Beer', 'American Amber Ale', 'American Brown Ale', 'American IPA', 'Altbier', 'Alternative Sugar Beer']

 Number of styles with multiples: 40
['American Amber Ale', 'American Barleywine', 'American Brown Ale', 'American IPA', 'American Pale Ale', 'American Stout', 'Baltic Porter', 'Belgian Blond Ale', 'Belgian Dark Strong Ale', 'Belgian Dubbel', 'Belgian Golden Strong Ale', 'Belgian Pale Ale', 'Belgian Tripel', 'Berliner Weisse', 'Bière de Garde', 'Blonde Ale', 'Cream Ale', 'Doppelbock', 'Eisbock', 'English Barleywine', 'English IPA', 'Flanders Red Ale', 'Foreign Extra Stout', 'Fruit Beer', 'Fruit Lambic', 'Gueuze', 'Irish Red Ale', 'Kölsch', 'Munich Dunkel', '

In [4]:
#need to change style names so they dont have '/' and ':' so we can save to csv
brewStyles = [sub.replace('/-', ' shilling') for sub in brewStyles] 
brewStyles = [sub.replace('/', '=') for sub in brewStyles]
brewStyles = [sub.replace(':', ';') for sub in brewStyles]

## Scrape search table for urls for each style
This is a dynamic webpage so we will have to use selenium.
First we will scrape the list of 181 unique styles, then we will scrape
the remaining 40 repeated styles afterwards since it requires slightly 
different code.

In [5]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys 

In [ ]:
#indicate if your running through multiples list (false for the first runthrough of the 181 unique
#and true on the second runthrough of the 40 multiples runthrough)
doingMultiples = True

driver = webdriver.Chrome()  
driver.get('https://www.brewersfriend.com/search/')

#close popup
xpath = "//*[@class='fancybox-item fancybox-close']"
WebDriverWait(driver, 40).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()

#find all search bars available
#searches[0] = method,..., searches[1,2,3,4,5,6] = [units,style,fermentables,hops,yeasts,other]
wait = WebDriverWait(driver,45)
xpath = "//*[@class='search']"
wait.until(EC.visibility_of_all_elements_located((By.XPATH,xpath)))
searchBars = driver.find_elements_by_xpath(xpath)

# xpath = "//*[@class='recipetitle']"
# recipe = driver.find_element_by_xpath(xpath)

#input brew method (only interested in 'All Grain' recipes for now - approx. 105k of them total)
searchBars[0].clear()                #clear any populated text (there isn't any in this case but its good practice)
searchBars[0].send_keys('All Grain') #input all grain into searchbar
searchBars[0].send_keys(Keys.ENTER)  #hit enter
time.sleep(3)
# wait.until(EC.staleness_of(recipe))

#iterate through each brewstyle and save urls to individual csv files - we can combine them all later
if doingMultiples == True:
    stylesToScrape = multiplesList
else:
    stylesToScrape = brewStyles
    
for style in stylesToScrape:  
    #input brew style
    searchBars[2].clear()
    time.sleep(1)
    searchBars[2].send_keys(style)
    time.sleep(1)
    if doingMultiples == True:
        searchBars[2].send_keys(Keys.DOWN)
        time.sleep(1)

    searchBars[2].send_keys(Keys.ENTER)
    time.sleep(6)

    recipeLinks = [] # initialize list to store recipeLinks

    #iterate through all of the pages and get all recipe links on each page
    #currentPage = 0 #initialize current page before entering while loop
    #emulate do-while loop
    while True:
        #find all recipe links in page
        xpath = "//*[@class='recipetitle']"
        wait.until(EC.visibility_of_element_located((By.XPATH,xpath)))
        recipes = driver.find_elements_by_xpath(xpath)

        #get links for each recipe
        for i in range(0,(len(recipes))):
            recipeLinks.append(recipes[i].get_attribute("href"))

        #go to next page
        time.sleep(random.random()*1+2)
        #click the last available next (to avoid clicking links with recipe names that have 'next' in them)
        #nexts = driver.find_elements_by_xpath("//*[contains(text(), 'Next')]") 
        nextButton = driver.find_elements_by_xpath("//a[@class=' recipe_search_nav' and text()= 'Next ›']")
        if len(nextButton) != 0:
            #nexts[len(nexts)-1].click() #click last available next is the correct next
            nextButton[0].click()
        else:
            break

        wait.until(EC.staleness_of(recipes[0]))

    #need to change style name so it doesnt have '/' or ':' so we can save to csv
    style = style.replace('/-', ' shilling').replace('/', '=').replace(':', ';')

    #save to csv
    print("Number of",style, "recipes: ", len(recipeLinks))
    df = pd.DataFrame(recipeLinks, columns = [style])
    if doingMultiples == True:
        df.to_csv(style+'2.csv', encoding = 'utf-8', index = False)
    else:
        df.to_csv(style+'.csv', encoding = 'utf-8', index = False)

# Scrape brew information using urls scraped in previous section

### Define scraping functions

In [6]:
def getBrewInfo(index):
    beerStats_container = soup.find('div', class_ = "description")
    if beerStats_container is not None:
        tableRows = beerStats_container.find_all('span', class_='viewStats')
        for i in range(0,len(tableRows)):
            if tableRows[i].find('span', class_='firstLabel').text == 'Method:':
                brewData['method'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Style:':
                brewData['style'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Boil Time:':
                brewData['boilTime'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Batch Size:':
                brewData['batchSize'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Pre Boil Size:':
                brewData['preBoilSize'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Post Boil Size:':
                brewData['postBoilSize'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Pre Boil Gravity:':
                if '°P' in tableRows[i].find('strong').text.strip():
                    dP = float(tableRows[i].find('strong').text.strip().split(' °P')[0])
                    brewData['preBoilGravity'][index] = str(round(1 + (dP/(258.6 - ((dP/258.2)*227.1))),3))
                else:
                    brewData['preBoilGravity'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Efficiency:':
                brewData['efficiency'][index] = tableRows[i].find('strong').text.strip().replace('%','')
            elif tableRows[i].find('span', class_='firstLabel').text == 'Source:':
                brewData['source'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Rating:':
                brewData['rating'][index] = tableRows[i].find('span',itemprop = "ratingValue").text.strip()
                brewData['numRating'][index] = tableRows[i].find('span',itemprop = "reviewCount").text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'No Chill:':
                brewData['noChill'][index] = ' '.join(tableRows[i].text.split()).split('No Chill: ',1)[1]
            elif tableRows[i].find('span', class_='firstLabel').text == 'Hop Utilization:':
                brewData['hopUtil'][index] = tableRows[i].find('strong').text.strip()
            elif tableRows[i].find('span', class_='firstLabel').text == 'Calories:':
                brewData['calories'][index] = tableRows[i].text.strip().split()[1].replace(',','')
            elif tableRows[i].find('span', class_='firstLabel').text == 'Carbs:':
                brewData['carbs'][index] = tableRows[i].text.strip().split()[1].replace(',','')
            elif tableRows[i].find('span', class_='firstLabel').text == 'URL:':
                brewData['links'][index] = tableRows[i].find('a').text.strip()
            else:
                print("Theres another general stat you're not aware of: ", url)

    return 

def getBrewInfo2(index):
    #need to convert degrees plato to specific gravity: SG = 1+ (dP / (258.6 – ( (dP/258.2) *227.1) ) )
    def platoToGravity(container, classValue):
        if '°P' in container.find('div', class_ = classValue).text.strip():
            dP = float(container.find('div', class_ = classValue).text.strip().split(' °P')[0])
            return str(round(1 + (dP/(258.6 - ((dP/258.2)*227.1))),3))
        else:
            return (container.find('div', class_ = classValue).text.strip())

    beerStatsOther_container = soup.find('div', class_='appbounds minstats')
    if beerStatsOther_container is not None:
        brewData['og'][index] = platoToGravity(beerStatsOther_container, 'value ogBatch')
        brewData['fg'][index] = platoToGravity(beerStatsOther_container, 'value fgBatch')       
        brewData['abv'][index] = beerStatsOther_container.find('div', class_='value abvMin').text.strip().replace('%','')
        brewData['ibu'][index] = beerStatsOther_container.find('div', class_='value ibuMin').text.strip()
        brewData['srm'][index] = beerStatsOther_container.find('div', class_='value srmMin').text.strip()
        
        if beerStatsOther_container.find('div', class_='value phMin').text.strip() != 'n/a':
            brewData['mashPh'][index] = beerStatsOther_container.find('div', class_='value phMin').text.strip()
      
    return 

def getFermInfo(index):
    fermAmount = []
    fermName   = [] 
    fermPPG    = [] #points per pound per gallon
    fermL      = [] #°L (degrees Lovibond)
    fermBillP  = [] #bill percentage

    ferm_container = soup.find('div', class_ = "brewpart", id = 'fermentables') #location of fermentables
    if ferm_container is not None: 
        tableRows = ferm_container.table.find_all('tr')
        # i = table row, j = table column 
        for i in range(1,len(tableRows)-1): #-1 to avoid last row (total weight)  
            for j in range(0,len(tableRows[i])):
                if j == 0: #amounts are in 1st column
                    fermAmount.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 1: #names in 2nd column...etc
                    #some recipes have the grain names hyperlinked, if hyperlinked there will be two
                    #text fields: the hop name and amount of hops
                    #e.g., this allows it to scrape just 'American - Wheat' instead of 'American - Wheat1.4 lb Wheat'
                    if (tableRows[i].find_all('td')[j].find('a') is None):
                        fermName.append(tableRows[i].find_all('td')[j].find('span').text.strip())
                    else:
                        fermName.append(tableRows[i].find_all('td')[j].find('a').text.strip())
                elif j == 2: #ppg
                    fermPPG.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 3: #°L (degrees Lovibond)
                    fermL.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 4: #bill percentage
                    fermBillP.append(tableRows[i].find_all('td')[j].text.strip().replace('%','')) 
 
    brewData['fermAmounts'][index] = fermAmount
    brewData['fermNames'][index] = fermName
    brewData['fermPPGs'][index] = fermPPG
    brewData['fermLs'][index] = fermL
    brewData['fermBillPs'][index] = fermBillP
    
    return 

def getHopInfo(index):
    hopAmount = []
    hopName = []
    hopForm = [] #i.e., leaf/whole or pellet
    hopAA = [] #alpha acid content
    hopUse = [] #when added i.e., during boil, dry hop, etc
    hopTime = []
    hopIBU = [] #international bitterness units
    hopBillP = [] #bill percentage

    hop_container = soup.find('div', class_ = "brewpart", id = 'hops')
    if hop_container is not None: #if hop table exists, get values
        tableRows = hop_container.table.find_all('tr')
        if hop_container.find('div', id='hopsSummary', style='display: none;') is not None: #if hops table isn't in summary view
            # i = table row, j = table column
            for i in range(1,len(tableRows)):   
                for j in range(0,len(tableRows[i])):
                    if j == 0: #amounts are in 1st column
                        hopAmount.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 1: #names in 2nd column...etc
                        #some recipes have the hop names hyperlinked, if hyperlinked there will be two
                        #text fields: the hop name and amount of hops
                        #e.g., this allows it to scrape just 'Citra' instead of 'Citra1 oz Citra Hops'
                        if tableRows[i].find_all('td')[j].find('a') is None:
                            hopName.append(tableRows[i].find_all('td')[j].find('span').text.strip())
                        else:
                            hopName.append(tableRows[i].find_all('td')[j].find('a').text.strip())
                    elif j == 2: #form
                        hopForm.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 3: #AA (degrees Lovibond)
                        hopAA.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 4: #Use
                        hopUse.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 5: #Time
                        hopTime.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 6: #IBU
                        hopIBU.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 7: #bill percentage
                        hopBillP.append(tableRows[i].find_all('td')[j].text.strip().replace('%',''))  
        else: #else hops is in summary view - smaller table
            for i in range(1,len(tableRows)-1):  #-1 since last row is total (dont need)
                for j in range(0,len(tableRows[i])): # i = table row, j = table column 
                    if j == 0: #amounts are in 1st column
                        hopAmount.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 1: 
                        #some recipes have the hop names hyperlinked, if hyperlinked there will be two
                        #text fields: the hop name and amount of hops
                        #e.g., this allows it to scrape just 'Citra' instead of 'Citra1 oz Citra Hops'
                        if tableRows[i].find_all('td')[j].find('a') is None:
                            #names and form are in 2nd column when hops are in summary view
                            unparsed = tableRows[i].find_all('td')[j].find('span').text.strip()
                            name_ = unparsed.split('(')[0].strip() #name_ has underscore because name is list of recipe names
                            form = unparsed.split('(')[1].replace(')','').strip()
                            hopName.append(name_)
                            hopForm.append(form)
                        else:
                            #names and form are in 2nd column when hops are in summary view
                            unparsed = tableRows[i].find_all('td')[j].find('a').text.strip()
                            name_ = unparsed.split('(')[0].strip()
                            form = unparsed.split('(')[1].replace(')','').strip()
                            hopName.append(name_)
                            hopForm.append(form)
                    elif j == 2: #IBU
                        hopIBU.append(tableRows[i].find_all('td')[j].text.strip())
                    elif j == 3: #bill percentage
                        hopBillP.append(tableRows[i].find_all('td')[j].text.strip()) 

                    #hop AA, use, and time aren't available in summarized table
                    hopAA.append('NA') #we can look up average AA content and add it later
                    hopUse.append('NA')
                    hopTime.append('NA')
                    
        brewData['hopAmounts'][index] = hopAmount
        brewData['hopNames'][index] = hopName
        brewData['hopForms'][index] = hopForm
        brewData['hopAAs'][index] = hopAA
        brewData['hopUses'][index] = hopUse
        brewData['hopTimes'][index] = hopTime
        brewData['hopIBUs'][index] = hopIBU
        brewData['hopBillPs'][index] = hopBillP

    return 

def getYeastInfo(index):
    yeastName = []
    yeastAmount = []
    yeastAttenuation = [] #yeast attenuation (avg)
    yeastOptTemp = [] #optimum temp
    yeastFloc = [] #flocculation
    yeastStarter = [] #binary yes or no
    yeastFermTemp = [] #fermentation temp
    yeastPitchRate = [] #pitch rate
    
    yeast_container = soup.find('div', class_ = "brewpart", id = 'yeasts')
    if yeast_container is not None:
        for i in range(0,len(yeast_container.find_all('tbody'))): #most recipes use 1 yeast but some have 2+
            yeastName.append(yeast_container.find_all('thead')[i].find('span').text.strip())
            #table is in a weird format compared to the others, also varies in dimensions
            #so we'll take in the entire text of the table and parse it
            unparsed = yeast_container.find_all('tbody')[i].find_all('tr')[0].find_all('td')[0].text
            #replace any whitespace with just 1 space
            unparsed = ' '.join(unparsed.split())
            #make it comma delimited so we can make a list of features
            unparsed = unparsed.replace('Attenuation', ', Attenuation').replace('Flocculation', ', Flocculation')
            unparsed = unparsed.replace('Optimum', ', Optimum').replace('Starter', ', Starter')
            unparsed = unparsed.replace('Fermentation', ', Fermentation').replace('Pitch', ', Pitch')
            unparsed = unparsed.split(',')

            #depending on what feature each element of the list is, assign respective value
            for i in range(0,len(unparsed)):
                if 'Amount:' in unparsed[i]:
                    yeastAmount.append(unparsed[i].split('Amount:')[1].strip())
                elif 'Attenuation (avg):' in unparsed[i]:
                    yeastAttenuation.append(unparsed[i].split('Attenuation (avg):')[1].strip()+' (average)')
                elif 'Attenuation (custom):' in unparsed[i]:
                    yeastAttenuation.append(unparsed[i].split('Attenuation (custom):')[1].strip()+' (custom)')
                elif 'Flocculation:' in unparsed[i]:
                    yeastFloc.append(unparsed[i].split('Flocculation:')[1].strip())
                elif 'Optimum Temp:' in unparsed[i]:
                    yeastOptTemp.append(unparsed[i].split('Optimum Temp:')[1].strip())
                elif 'Starter:' in unparsed[i]:
                    yeastStarter.append(unparsed[i].split('Starter:')[1].strip())
                elif 'Fermentation Temp:' in unparsed[i]:
                    yeastFermTemp.append(unparsed[i].split('Fermentation Temp:')[1].strip())
                elif 'Pitch Rate:' in unparsed[i]:
                    yeastPitchRate.append(unparsed[i].split('Pitch Rate:')[1].strip())
                else:
                    print("Theres a yeast feature you're unaware of: ", link)

        brewData['yeastNames'][index] = yeastName
        brewData['yeastAmounts'][index] = yeastAmount
        brewData['yeastAttenuations'][index] = yeastAttenuation
        brewData['yeastOptTemps'][index] = yeastOptTemp
        brewData['yeastFlocs'][index] = yeastFloc
        brewData['yeastStarters'][index] = yeastStarter
        brewData['yeastFermTemps'][index] = yeastFermTemp
        brewData['yeastPitchRates'][index] = yeastPitchRate
  
    return 

def getOtherInfo(index):
    otherAmount = []
    otherName = []
    otherType = []
    otherUse = []
    otherTime = []

    other_container = soup.find('div', class_ = "brewpart", id = 'others')
    if other_container is not None:
        tableRows = other_container.table.find_all('tr')
        # i = table row, j = table column
        for i in range(1,len(tableRows)):  
            for j in range(0,len(tableRows[i])):
                if j == 0: #amounts are in 1st column
                    otherAmount.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 1: #names in 2nd column...etc
                    #similar problem here as with hopNames and fermNames, see them for more details
                    if tableRows[i].find_all('td')[j].find('a') is None:
                        otherName.append(tableRows[i].find_all('td')[j].find('span').text.strip())
                    else:
                        otherName.append(tableRows[i].find_all('td')[j].find('a').text.strip())
                elif j == 2: #type
                    otherType.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 3: #use
                    otherUse.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 4: #time
                    otherTime.append(tableRows[i].find_all('td')[j].text.strip())
    
        brewData['otherAmounts'][index] = otherAmount
        brewData['otherNames'][index] = otherName
        brewData['otherTypes'][index] = otherType
        brewData['otherUses'][index] = otherUse
        brewData['otherTimes'][index] = otherTime
    
    return 

def getWaterInfo(index):
    water_container = soup.find('div',class_='brewpart',id='water')
    if water_container is not None:
        brewData['ca'][index] = water_container.find('table').find_all('td')[0].text.strip()
        brewData['mg'][index] = water_container.find('table').find_all('td')[1].text.strip()
        brewData['na'][index] = water_container.find('table').find_all('td')[2].text.strip()
        brewData['cl'][index] = water_container.find('table').find_all('td')[3].text.strip()
        brewData['so4'][index] = water_container.find('table').find_all('td')[4].text.strip()
        brewData['hco3'][index] = water_container.find('table').find_all('td')[5].text.strip()

        #check if water notes are available, its the first water criteria that spans 6 columns
        #the one after it is a link to mash chemistry and brewing water calculator
        notes_container = water_container.find('table').find('td',colspan="6")
        if (notes_container is not None 
            and notes_container.text.strip()!='Mash Chemistry and Brewing Water Calculator'):
            brewData['twNotes'][index] = notes_container.text.strip()
            
    return

def getMashInfo(index):
    mashAmount = []
    mashDesc = []
    mashType = []
    mashTemp = []
    mashTime = []
    
    mash_container = soup.find('div', class_ = "brewpart mash_guidelines", id = 'mashsteps')
    if mash_container is not None:
        #check to see if mash thickness is present, it's in the last row and spans all columns
        if mash_container.table.find('tr', class_='bf_table_row_totals') is None: 
            k = 0
        else:
            k = 1
        # i = table row, j = table column 
        tableRows = mash_container.table.find_all('tr')
        for i in range(1,len(tableRows)-k): #-k so we dont loop over last row if mashThickness is present
            for j in range(0,len(tableRows[i])):
                if j == 0: #amounts are in 1st column
                    mashAmount.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 1: #description
                    if tableRows[i].find_all('td')[j].text.strip() == '':
                        mashDesc.append('NA')
                    else:
                        mashDesc.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 2: #type
                    if tableRows[i].find_all('td')[j].text.strip() == '':
                        mashType.append('NA')
                    else:
                        mashType.append(tableRows[i].find_all('td')[j].text.strip())
                elif j == 3: #temp
                    mashTemp.append(tableRows[i].find_all('td')[j].text.strip().replace('--','NA'))
                elif j == 4: #time
                    mashTime.append(tableRows[i].find_all('td')[j].text.strip().replace('--','NA'))
        
        if k==1: #parse out thickness number and units
            wordList = tableRows[len(tableRows)-1].text.strip().split()
            thick = wordList[3]+' '+wordList[4]
            brewData['mashThick'][index] = thick

        brewData['mashAmounts'][index] = mashAmount
        brewData['mashDescs'][index] = mashDesc
        brewData['mashTypes'][index] = mashType
        brewData['mashTemps'][index] = mashTemp
        brewData['mashTimes'][index] = mashTime

    return 

def getRemainingInfo(index): 
    brewData['recipeURL'][idx] = url
    
    #get name of recipe
    name_container = soup.find('h3', itemprop='name')
    if name_container is not None:
        brewData['name'][index] = name_container.text.strip()

    #get author of recipe
    author_container = soup.find('span', itemprop = "author")
    if author_container is not None:
        brewData['author'][index] = author_container.text.strip()

    #get date created
    date_container = soup.find('div', class_="extra") 
    if date_container is not None:
        brewData['dateCreated'][index] = date_container.text.strip().replace('Created', '')
    
        #get priming info
    priming_container = soup.find('div', class_='brewpart',  id = "primingmethod")
    if priming_container is not None:
        brewData['priming'][index] = priming_container.find('td').text.strip().replace('\xa0','').replace('\t','').replace('\n',',')
    
    #get recipe notes 
    notes_container = soup.find('div',class_='ui message')
    if notes_container is not None:
        brewData['notes'][index] = notes_container.text.strip()

    #get number of views recipe has and number of times other users have brewed recipe
    viewsAndBrews_container = soup.find('div', class_='ui two statistics')
    if viewsAndBrews_container is not None:
        brewData['views'][index] = viewsAndBrews_container.find_all('div',class_='value')[0].text.strip()
        brewData['brews'][index] = viewsAndBrews_container.find_all('div',class_='value')[1].text.strip()
        
    return

### Scrape information from urls for each style

In [18]:
brewStyles = sorted(list(set(brewStyles)))
if doingMultiples == True:
    stylesToScrape = multiplesList
else:
    stylesToScrape = brewStyles

for style in stylesToScrape[0:3]:
    start_time = time.time()
    if doingMultiples == True:
        data = pd.read_csv(style+'2.csv', encoding = "utf-8")
    else:
        data = pd.read_csv(style+'.csv', encoding = "utf-8")

    brewFeatures = ['recipeURL','name','author','method','style','boilTime','batchSize','preBoilSize',
                    'postBoilSize','preBoilGravity','efficiency','source','noChill','rating','numRating',
                    'hopUtil','calories','carbs','views','brews','links','dateCreated','og','fg','abv',
                    'ibu','srm','mashPh','fermNames','fermAmounts','fermPPGs','fermLs','fermBillPs',
                    'hopAmounts','hopNames','hopForms','hopAAs','hopUses','hopTimes','hopIBUs','hopBillPs',
                    'yeastNames','yeastAmounts','yeastAttenuations','yeastOptTemps','yeastFlocs',
                    'yeastStarters','yeastFermTemps','yeastPitchRates','ca','mg','na','cl','so4','hco3',
                    'twNotes','priming', 'otherNames','otherAmounts','otherTypes','otherUses','otherTimes',
                    'mashAmounts','mashDescs','mashTypes','mashTemps','mashTimes','mashThick','notes']

    #initialize dictionary of features
    brewData = {feature: [] for feature in brewFeatures}

    #loop through each recipe
    idx = -1
    for link in data[style]:

        url = link
        hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
        req = Request(url, headers=hdr)

        try: #try to open url
            page = urlopen(req)
        except HTTPError as e:
            print('The server couldn\'t fulfill the request.')
            print('Error code: ', e.code)
            print(link)
            sys.exit(1)
        except URLError as e:
            print('We failed to reach a server.')
            print('Reason: ', e.reason)
            print(link)
            sys.exit(1)
        else:
            idx = idx+1   
            soup = BeautifulSoup(page, "lxml")

            #initialize all features as 'NA'
            for feature in brewData:
                brewData[feature].append('NA')

            #get all info, change 'NA's to correct values only if theyre present
            getBrewInfo(idx)
            getBrewInfo2(idx)
            getFermInfo(idx)
            getMashInfo(idx)
            getWaterInfo(idx)
            getOtherInfo(idx)
            getYeastInfo(idx)
            getHopInfo(idx)
            getRemainingInfo(idx)
            
            # pause loop to avoid overloading server/ getting banned
            time.sleep(random.randint(1,2)+1)

    df = pd.DataFrame(brewData)      

    if doingMultiples == True:
        df.to_csv(style+'2.csv', encoding = 'utf-8', index = False)
    else:
        df.to_csv(style+'.csv', encoding = 'utf-8', index = False)
        
    print(style,'('+str(len(data[style]))+')',"--- %s seconds ---" % (time.time() - start_time))

American Amber Ale (1615) --- 5890.752970933914 seconds ---
American Barleywine (223) --- 785.9158563613892 seconds ---
American Brown Ale (780) --- 2820.154831647873 seconds ---


## Merge all style csv's into one file
We will now merge all individual style csv files into one master file

In [3]:
###  Identify Styles and Style Multiples
brewStyles = list(pd.read_csv('stylesList.csv',encoding='utf-8')['Styles'])
print('Number of styles (includes multiples):',len(brewStyles))
print(list(brewStyles[0:20]))

multiplesList = brewStyles.copy()
for x in list(set(brewStyles)):
    multiplesList.remove(x)

print('\n','Number of styles with multiples:',len(multiplesList))
print(multiplesList)

uniqueStyles = sorted(list(set(brewStyles)))
print('\n','Number of unique styles:',len(uniqueStyles))
print(uniqueStyles[0:20])

#need to change style names so they dont have '/' and ':' so we can save to csv
brewStyles = [sub.replace('/-', ' shilling') for sub in brewStyles] 
brewStyles = [sub.replace('/', '=') for sub in brewStyles]
brewStyles = [sub.replace(':', ';') for sub in brewStyles]

#need to change style names so they dont have '/' and ':' so we can save to csv
uniqueStyles = [sub.replace('/-', ' shilling') for sub in uniqueStyles] 
uniqueStyles = [sub.replace('/', '=') for sub in uniqueStyles]
uniqueStyles = [sub.replace(':', ';') for sub in uniqueStyles]

Number of styles (includes multiples): 221
['Altbier', 'Alternative Grain Beer', 'Alternative Sugar Beer', 'American Amber Ale', 'American Amber Ale', 'American Barleywine', 'American Barleywine', 'American Brown Ale', 'American Brown Ale', 'American IPA', 'American IPA', 'American Lager', 'American Light Lager', 'American Pale Ale', 'American Pale Ale', 'American Porter', 'American Stout', 'American Stout', 'American Strong Ale', 'American Wheat Beer']

 Number of styles with multiples: 40
['American Amber Ale', 'American Barleywine', 'American Brown Ale', 'American IPA', 'American Pale Ale', 'American Stout', 'Baltic Porter', 'Belgian Blond Ale', 'Belgian Dark Strong Ale', 'Belgian Dubbel', 'Belgian Golden Strong Ale', 'Belgian Pale Ale', 'Belgian Tripel', 'Berliner Weisse', 'Bière de Garde', 'Blonde Ale', 'Cream Ale', 'Doppelbock', 'Eisbock', 'English Barleywine', 'English IPA', 'Flanders Red Ale', 'Foreign Extra Stout', 'Fruit Beer', 'Fruit Lambic', 'Gueuze', 'Irish Red Ale', 'Köls

In [4]:
#identify styles there arent any csv's for (i.e., the styles there were no all-grain recipes for)
import os
import glob

path = r'C:\Users\Theron\Documents\Brewscrape\Styles'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(file.replace('.csv',''))
    
list(set(uniqueStyles)-set(files))

['Common Perry', 'New Zealand Pilsner', 'Traditional Perry']

In [5]:
#remove styles that had no all grain recipes: common perry, new zealand pilsner
uniqueStyles.remove('Common Perry')
uniqueStyles.remove('Traditional Perry')
uniqueStyles.remove('New Zealand Pilsner')

In [18]:
path = r'C:\Users\Theron\Documents\Brewscrape\Styles'           # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent
#remove styles list
all_files.remove('C:\\Users\\Theron\\Documents\\Brewscrape\\Styles\\stylesList.csv')

df_from_each_file = (pd.read_csv(f) for f in all_files)
data = pd.concat(df_from_each_file, ignore_index=True)
#drop rows that have all na's, in this case if method is na, the rest are as well except for recipeURL
data = data.dropna(axis=0,how='all',subset=['method'])
#drop recipes that use method other than all grain (BIAB, extract, etc)
data = data[data.method == 'All Grain']

#do some cleaning before saving
data['views'] = [float(str(x).replace(',','')) for x in data.views]
data['boilTime'] = [float(x.split(' min')[0]) for x in data.boilTime]

#convert liters to gallons
#there are 0.265172 gallons per liter
data['batchSize'] = list(map(lambda x: round(float(x.split(' ')[0])*0.264172,2) if x.split(' ')[1]=='liters' 
                                                else float(x.split(' ')[0]), data.batchSize))
data['preBoilSize'] = list(map(lambda x: round(float(x.split(' ')[0])*0.264172,2) if x.split(' ')[1]=='liters' 
                                                else float(x.split(' ')[0]), data.preBoilSize))
data['postBoilSize'] = list(map(lambda x: round(float(x.split(' ')[0])*0.264172,2) 
                                if (type(x)!=float and x.split(' ')[1]=='liters')
                                else (float(x.split(' ')[0] if (type(x)!=float and x.split(' ')[1]=='gallons')
                                            else float(x))), data.postBoilSize))
data.to_csv('allStylesData.csv',encoding = 'utf-8',index = False)

C:\Users\Theron\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  import sys


# Scrape ingredient information provided on brewersfriend
This information will be used to clean the ingredients to be used in the beer classifier and recipe generator. <br>
https://www.brewersfriend.com/hops/ <br> https://www.brewersfriend.com/fermentables/ <br> https://www.brewersfriend.com/yeasts/ <br> https://www.brewersfriend.com/other/

## Scrape hop information
(will eventually make a function that will scrape all 4 ingredient types using same code)

In [19]:
#get list of styles we will iterate through to scrape urls
url = 'https://www.brewersfriend.com/hops/'
hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
req = Request(url, headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, "lxml")

hop_table =  soup.find('table', class_='browse-results').find('tbody')
hop_rows = hop_table.find_all('tr')

hop_features = ['hop','type','avg_AA','use','recipe_nums']
hop_data = {feature: [] for feature in hop_features}

for i in range(0,len(hop_rows)):
    hop_data['hop'].append(hop_rows[i].find_all('td')[0].text) #name is in 1st column
    hop_data['type'].append(hop_rows[i].find_all('td')[1].text)
    hop_data['avg_AA'].append(hop_rows[i].find_all('td')[2].text)
    hop_data['use'].append(hop_rows[i].find_all('td')[3].text)
    hop_data['recipe_nums'].append(hop_rows[i].find_all('td')[4].text) #num of recipes is in 5th column


hop_df = pd.DataFrame(hop_data)
print(len(hop_df))
print(hop_df[0:5])
hop_df.to_csv('hop_info.csv',encoding = 'utf-8',index = False)

438
        hop        type avg_AA   use recipe_nums
0   Admiral      Pellet   14.4  Boil       3,755
1   Ahtanum      Pellet    5.4  Boil      15,746
2  Amarillo      Pellet    8.6  Boil     195,778
3    Apollo        Plug   18.9  Boil      12,469
4    Aquila  Leaf/Whole    6.7  Boil          75


## Scrape fermentable information

In [20]:
#get list of styles we will iterate through to scrape urls
url = 'https://www.brewersfriend.com/fermentables/'
hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
req = Request(url, headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, "lxml")

ferm_table =  soup.find('table', class_='browse-results').find('tbody')
ferm_rows = ferm_table.find_all('tr')

ferm_features = ['fermentable','country','category','type',
                 'color','ppg','recipe_nums']
ferm_data = {feature: [] for feature in ferm_features}

for i in range(0,len(ferm_rows)):
    ferm_data['fermentable'].append(ferm_rows[i].find_all('td')[0].text) #name is in 1st column
    ferm_data['country'].append(ferm_rows[i].find_all('td')[1].text) 
    ferm_data['category'].append(ferm_rows[i].find_all('td')[2].text)
    ferm_data['type'].append(ferm_rows[i].find_all('td')[3].text) 
    ferm_data['color'].append(ferm_rows[i].find_all('td')[4].text) 
    ferm_data['ppg'].append(ferm_rows[i].find_all('td')[5].text)
    ferm_data['recipe_nums'].append(ferm_rows[i].find_all('td')[6].text) #num of recipes is in 7th column

ferm_df = pd.DataFrame(ferm_data)
print(len(ferm_df))
print(ferm_df[0:5])
ferm_df.to_csv('ferm_info.csv',encoding = 'utf-8',index = False)

845
       fermentable  country category             type   color  ppg recipe_nums
0       Abbey Malt   German    Grain        Base malt  17 °L    33       3,089
1          Acerola             Fruit            Fruit   0 °L   1.5           5
2  Acidulated Malt   German    Grain  Acidulated malt   3 °L    27      50,606
3     Agave Nectar             Sugar            Sugar   2 °L    35         305
4         Ale Malt  Ireland    Grain        Base malt   3 °L    37       1,081


## Scrape yeast information

In [22]:
#get list of styles we will iterate through to scrape urls
url = 'https://www.brewersfriend.com/yeasts/'
hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
req = Request(url, headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, "lxml")

yeast_table =  soup.find('table', class_='browse-results').find('tbody')
yeast_rows = yeast_table.find_all('tr')

yeast_features = ['yeast','lab','code','type','alcohol_tolerance','flocculation',
                  'attenuation','min_temp','max_temp']
yeast_data = {feature: [] for feature in yeast_features}

for i in range(0,len(yeast_rows)):
    yeast_data['yeast'].append(yeast_rows[i].find_all('td')[0].text) #name is in 1st column
    yeast_data['lab'].append(yeast_rows[i].find_all('td')[1].text) 
    yeast_data['code'].append(yeast_rows[i].find_all('td')[2].text)
    yeast_data['type'].append(yeast_rows[i].find_all('td')[3].text) 
    yeast_data['alcohol_tolerance'].append(yeast_rows[i].find_all('td')[4].text) 
    yeast_data['flocculation'].append(yeast_rows[i].find_all('td')[5].text)
    yeast_data['attenuation'].append(yeast_rows[i].find_all('td')[6].text) 
    yeast_data['min_temp'].append(yeast_rows[i].find_all('td')[7].text)
    yeast_data['max_temp'].append(yeast_rows[i].find_all('td')[8].text) #max temp is in 9th column
    
yeast_df = pd.DataFrame(yeast_data)
print(len(yeast_df))
print(yeast_df[0:5])
yeast_df.to_csv('yeast_info.csv',encoding = 'utf-8',index = False)

576
                           yeast lab   code  type alcohol_tolerance  \
0     - Saccharomycodes ludwigii             Ales              High   
1                    Coopers Ale   -          Ale            Medium   
2                      Doric Ale   -          Ale            Medium   
3                       Edme Ale   -          Ale            Medium   
4  Gervin Ale (by Muntons) GV-12   -  GV-12   Ale               n/a   

  flocculation attenuation min_temp max_temp  
0          Low          0%      0°F      0°F  
1         High         72%     68°F     80°F  
2       Medium         74%     62°F     72°F  
3       Medium         74%     62°F     70°F  
4         High         77%     57°F     70°F  


## Scrape other ingredient information

In [23]:
#get list of styles we will iterate through to scrape urls
url = 'https://www.brewersfriend.com/other/'
hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
req = Request(url, headers=hdr)
page = urlopen(req)
soup = BeautifulSoup(page, "lxml")

other_table =  soup.find('table', class_='browse-results').find('tbody')
other_rows = other_table.find_all('tr')

other_features = ['ingredient','type','use','num_recipes']
other_data = {feature: [] for feature in other_features}

for i in range(0,len(other_rows)):
    other_data['ingredient'].append(other_rows[i].find_all('td')[0].text) #name is in 1st column
    other_data['type'].append(other_rows[i].find_all('td')[1].text) 
    other_data['use'].append(other_rows[i].find_all('td')[2].text)
    other_data['num_recipes'].append(other_rows[i].find_all('td')[3].text) 
    
other_df = pd.DataFrame(other_data)
print(len(other_df))
print(other_df[0:5])
other_df.to_csv('other_info.csv',encoding = 'utf-8',index = False)

1593
                    ingredient       type   use num_recipes
0                     Cinnamon      Spice  Boil          37
1    Servomyces Yeast Nutrient      Other  Boil          28
2                10% Phos Acid  Water Agt  Mash          52
3  10% Phos Acid To Sparge 5.4  Water Agt  Mash          31
4          10% Phosphoric Acid  Water Agt  Mash          91
